In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

symbols = ["EURUSD", "USDJPY", "GBPUSD", "XAUUSD"]
timeframe = mt5.TIMEFRAME_D1
num_bars = 500
rolling_window = 30
sharpe_window = 60
confidence_level = 0.95


def rolling_sharpe(returns, window, risk_free_rate=0.0):
    mean_ret = returns.rolling(window).mean()
    std_ret = returns.rolling(window).std()
    sharpe = (mean_ret - risk_free_rate) / std_ret * np.sqrt(252)
    return sharpe


def historical_var(returns, confidence=confidence_level):
    return returns.quantile(1 - confidence)


def historical_cvar(returns, confidence=confidence_level):
    var = historical_var(returns, confidence)
    return returns[returns <= var].mean()


def fetch_and_calc(symbol):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    if rates is None:
        print(f"Failed to get rates for {symbol}")
        return None

    df = pd.DataFrame(rates)
    df['Date'] = pd.to_datetime(df['time'], unit='s')
    df.rename(columns={'close': 'Price'}, inplace=True)
    df = df[['Date', 'Price']]
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)

    df['DailyReturn'] = df['Price'].pct_change()
    df.dropna(inplace=True)

    df['CumulativeReturn'] = (1 + df['DailyReturn']).cumprod() - 1
    df['CumMax'] = df['CumulativeReturn'].cummax()
    df['Drawdown'] = df['CumulativeReturn'] - df['CumMax']
    df['RollingVolatility'] = (
        df['DailyReturn'].rolling(rolling_window).std() * np.sqrt(252)
    )
    df['RollingSharpe'] = rolling_sharpe(df['DailyReturn'], sharpe_window)
    df['VaR_95'] = df['DailyReturn'].rolling(rolling_window).apply(
        lambda x: historical_var(x, confidence_level)
    )
    df['CVaR_95'] = df['DailyReturn'].rolling(rolling_window).apply(
        lambda x: historical_cvar(x, confidence_level)
    )

    return df


if not mt5.initialize():
    print("MT5 initialization failed, error code =", mt5.last_error())
    quit()


dfs = {}
for sym in symbols:
    dfs[sym] = fetch_and_calc(sym)

mt5.shutdown()

fig = go.Figure()

# For toggling visibility - total traces = len(symbols)*6
# We'll show traces for the first symbol initially
visibility = []
for i, _ in enumerate(symbols):
    is_visible = (i == 0)
    for _ in range(6):
        visibility.append(is_visible)

# Add traces for all symbols
for i, sym in enumerate(symbols):
    df = dfs[sym]
    if df is None:
        # Add empty traces so indexes match
        for metric in [
            "CumulativeReturn",
            "Drawdown",
            "RollingVolatility",
            "RollingSharpe",
            "VaR_95",
            "CVaR_95",
        ]:
            fig.add_trace(
                go.Scatter(
                    x=[],
                    y=[],
                    mode="lines",
                    name=f"{sym} {metric}",
                    visible=False,
                )
            )
        continue

    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['CumulativeReturn'],
            mode='lines',
            name=f"{sym} Cumulative Return",
            visible=visibility[i * 6 + 0]
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['Drawdown'],
            mode='lines',
            name=f"{sym} Drawdown",
            visible=visibility[i * 6 + 1]
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['RollingVolatility'],
            mode='lines',
            name=f"{sym} Rolling Volatility",
            visible=visibility[i * 6 + 2]
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['RollingSharpe'],
            mode='lines',
            name=f"{sym} Rolling Sharpe",
            visible=visibility[i * 6 + 3]
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['VaR_95'],
            mode='lines',
            name=f"{sym} VaR 95%",
            visible=visibility[i * 6 + 4]
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['CVaR_95'],
            mode='lines',
            name=f"{sym} CVaR 95%",
            line=dict(dash='dash'),
            visible=visibility[i * 6 + 5]
        )
    )

# Create dropdown menu buttons for symbols
buttons = []
for i, sym in enumerate(symbols):
    # Build visibility array for each symbol
    vis = [False] * len(symbols) * 6
    for j in range(6):
        vis[i * 6 + j] = True
    buttons.append(
        dict(
            label=sym,
            method="update",
            args=[
                {"visible": vis},
                {"title": f"Overlaid Risk Metrics for {sym}"},
            ],
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=0.1,
            y=1.15,
            xanchor='left',
            yanchor='top',
        )
    ],
    legend_title="Metrics",
    height=700,
    width=1100,
    hovermode="x unified",
    xaxis_title="Date",
    yaxis_title="Value",
)

pio.write_image(fig, "risk_dashboard.png", width=1100, height=700)

fig.show()
